# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)
#### Aluno: Bruno Crespo Ferreira

## Problema

Suponha que temos uma tabela de nutrientes de diferentes tipos de alimentos. Sabendo o valor diário de referência (VDR) de cada nutriente e o preço de cada unidade de alimento, qual é a dieta ótima que contém pelo menos o valor diário de referência e seja de menor custo?

**Observação**: embora o problema possa ser resolvido com otimização inteira, optou-se por manter as variáveis contínuas.


## 1) Modelagem Matemática e Solução Ótima

Para esta modelagem, considere $m$ nutrientes e $n$ alimentos. Defina $a_{ij}$ a quantidade do nutriente $i$ no alimento $j$, e $r_{i}$ o valor diário mínimo de referência do nutriente $i$. Adicionalmente, seja $c_{j}$ o preço do alimento $j$.

Sua implementação do modelo de programação linear, utilizando o *gurobipy*, deve estabelecer as quantidades (em gramas) de cada alimento $j$ na dieta. Como inputs, utilizaremos dados do McDonalds, já disponíveis no Campus Virtual.

Os valores dos nutrientes são em % do VDR, por isso podemos usar simplesmente $r_{i} = 100\%$. As colunas que contém um "NA" indicam que o valor "default" é usado. Logo, se "NA" aparece em um parâmetro que reflete um limite inferior, deve-se adotar o valor zero (0) como "default". Caso "NA" apareça em parâmetros de limite superior, então o valor "default" será um inteiro suficientemente grande (teoricamente: ∞).

**Observação**: Embora o enunciado afirme que os valores dos nutrientes são em %, a implementação considera a interpretação de que o VDR está informado em gramas (essa diferença não muda o resultado, somente o expande para considerar utilizar o VDR máximo).

### Parâmetros

* $m$ nutrientes;
* $n$ alimentos;
* $a_{ij}$: quantidade do nutriente $i$ no alimento $j$;
* $r_{i}$: VDR mínimo do nutriente $i$;
* $s_{i}$: VDR máximo do nutriente $i$;
* $c_{j}$: preço do alimento $j$;
* $f_{min, j}$: limite inferior de compra do alimento $j$;
* $f_{max, j}$: limite superior de compra do alimento $j$;
* $veg_{j}$: o alimento $j$ é vegetariano (1) ou não (0).


### Variáveis

* $x_{j}$: quantidade do alimento $j$ em gramas.

### Função Objetivo

Consiste em **minimizar** o custo total dos alimentos da dieta.
\begin{equation}
\sum_{j}^n c_{j}x_{j}
\end{equation}

### Restrições

* Restrição 1: A dieta ótima deve conter, pelo menos, o VDR mínimo ($r_{i}$) de cada nutriente.
\begin{equation}
\sum_{j}^n a_{ji}x_{j} \ge r_{i}, \quad \forall i \in \{1,2,...,m\}.
\end{equation}

* Restrição 2: A dieta ótima não pode ultrapassar o VDR máximo ($s_{i}$) de cada nutriente.
\begin{equation}
\sum_{j}^n a_{ji}x_{j} \le s_{i}, \quad \forall i \in \{1,2,...,m\}.
\end{equation}

* Restrição 3: A compra do alimento $j$ deve estar entre os limites inferiores ($f_{min,j}$) e superiores ($f_{max,j}$) de compra.
\begin{equation}
f_{min,j} \le x_{j} \le f_{max,j} \quad \forall j \in \{1,2,...,n\}
\end{equation}


### Modelo

\begin{equation}
\sum_{j}^n c_{j}x_{j}
\end{equation}

sujeito a:

\begin{alignat}{2}
\sum_{j}^n a_{ji}x_{j} \ge r_{i}, \quad \forall i \in \{1,2,...,m\}. \\
\sum_{j}^n a_{ji}x_{j} \le s_{i}, \quad \forall i \in \{1,2,...,m\}. \\
x_{j} \ge f_{min,j} \quad \forall j \in \{1,2,...,n\} \\
x_{j} \le f_{max,j} \quad \forall j \in \{1,2,...,n\}
\end{alignat}

Considerando que $f_{min,j}$ e $f_{max,j}$ sempre são não negativos.

### Instalação da biblioteca Gurobi

In [ ]:
!pip install gurobipy

### Declaração do objeto que representa o modelo matemático

In [ ]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np
import re
import os

modelo = Model("MacDonalds_Dieta")

Restricted license - for non-production use only - expires 2026-11-23


### Leitura e formatação dos arquivos

**Observação**: é de responsabilidade do leitor inserir os arquivos de dados para a análise, e se necessário, alterar o diretorio e/ou nomes dos arquivos nos códigos.

In [ ]:
# Função que substitui as tabulações (\t) por espaços em branco dentro de strings entre aspas.
def substituir_tab_entre_aspas(match):
  return match.group(0).replace('\t', ' ')

In [ ]:
# Função que arruma a formatação de um arquivo, substituindo espaços por tabulações e ajustando múltiplas tabulações.
def arrumar_formatacao(diretorio, nome_arquivo):
  caminho_arquivo = os.path.join(diretorio, nome_arquivo)
  with open(caminho_arquivo, 'r', encoding='utf-8') as f:
    conteudo = f.read()

  # Substitui todos os espaços por tabulações (\t).
  conteudo_limpo = re.sub(r' ', '\t', conteudo)
  # Substitui múltiplas tabulações consecutivas por uma única tabulação.
  conteudo_limpo = re.sub(r'\t+', '\t', conteudo_limpo)
  conteudo_limpo = re.sub(r'"[^"]*"', substituir_tab_entre_aspas, conteudo_limpo)

  novo_arquivo = os.path.join(diretorio, f"formatado_{nome_arquivo}")
  with open(novo_arquivo, 'w', encoding='utf-8') as f:
    f.write(conteudo_limpo)
  return novo_arquivo

In [ ]:
nutrientes = pd.read_csv(arrumar_formatacao("/content/", "McDonalds-nutr.wsv"), sep='\t', skiprows=1)
nutrientes.columns = ['Nutriente', 'VDR_min', 'VDR_max']

alimentos = pd.read_csv(arrumar_formatacao("/content/", "McDonalds-food.wsv"), sep="\t", skiprows=2)
alimentos.columns = ['Alimento', 'Custo', 'Compra_min', 'Compra_max', 'Veg']

quantidades = pd.read_csv(arrumar_formatacao("/content/", "McDonalds-amnt.wsv"), sep="\t", skiprows=1)
quantidades.rename(columns={quantidades.columns[0]: 'Alimento'}, inplace=True)

#### Substituindo NaN para valores default

In [ ]:
nutrientes['VDR_min'] = nutrientes['VDR_min'].fillna(0).astype(float)
nutrientes['VDR_max'] = nutrientes['VDR_max'].fillna(np.inf).astype(float)

alimentos['Compra_min'] = alimentos['Compra_min'].fillna(0).astype(float)
alimentos['Compra_max'] = alimentos['Compra_max'].fillna(np.inf).astype(float)

### Obtendo os parâmetros

In [ ]:
alimentos_nomes = quantidades['Alimento'].values
coeficientes_colunas = quantidades.columns[1:]
coeficientes = quantidades[coeficientes_colunas].values

custos = alimentos['Custo'].values
lim_inf_compra = alimentos['Compra_min'].values
lim_sup_compra = alimentos['Compra_max'].values
eh_vegano = alimentos['Veg'].values

nutrientes_nomes = nutrientes['Nutriente'].values
vdr_min = nutrientes['VDR_min'].values
vdr_max = nutrientes['VDR_max'].values

### Variável de decisão

In [ ]:
x = {j: modelo.addVar(name=f"{alimentos_nomes[j]}", vtype=GRB.CONTINUOUS, lb=lim_inf_compra[j], ub=lim_sup_compra[j]) for j in range(len(alimentos_nomes))}

### Função objetivo

In [ ]:
modelo.setObjective(quicksum(custos[j] * x[j] for j in range(len(alimentos_nomes))), GRB.MINIMIZE)

### Restrições

In [ ]:
for i, nutriente in enumerate(nutrientes_nomes):
  modelo.addConstr(quicksum(coeficientes[j][i] * x[j] for j in range(len(alimentos_nomes))) >= vdr_min[i], f"Min_{nutriente}")
  if vdr_max[i] < float('inf'): # Apenas adiciona a restrição de máximo se existir (diferente de infinito)
    modelo.addConstr(quicksum(coeficientes[j][i] * x[j] for j in range(len(alimentos_nomes))) <= vdr_max[i], f"Max_{nutriente}")

### Resolvendo o problema

In [ ]:
modelo.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 84 columns and 634 nonzeros
Model fingerprint: 0x8971e040
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 0 rows and 16 columns
Presolve time: 0.01s
Presolved: 10 rows, 68 columns, 580 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.500000e+01   0.000000e+00      0s
       3    2.4311927e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.431192661e+01


### Imprimindo as soluções do problema

In [ ]:
if modelo.status == GRB.OPTIMAL:
  print("\nSolução ótima encontrada!")
  for j in range(len(alimentos_nomes)):
    if x[j].x > 1e-6:  # Apenas exibe valores significativos
      print(f"{x[j].varName}: {x[j].x:.2f} unidades")
  print(f"Custo total: R$ {modelo.objVal:.2f}")
else:
    print("Nenhuma solução ótima encontrada.")


Solução ótima encontrada!
McDuplo: 5.05 unidades
Casquinha Baunilha: 2.29 unidades
Maçã: 3.21 unidades
Custo total: R$ 24.31


## 2) Análise de Sensibilidade

In [ ]:
print("\nIntervalos de Custos (Vetor de Custos):")
for variavel in modelo.getVars():
  if variavel.x > 1e-6:  # Apenas exibe valores significativos
    print(f"Variável {variavel.VarName}:")
    print(f"  lower bound do custo (SAObjLow): {variavel.SAObjLow}")
    print(f"  upper bound do custo (SAObjUp): {variavel.SAObjUp}")

print("\nIntervalos de Recursos (Vetor de Recursos):")
for restricao in modelo.getConstrs():
  print(f"Restrição {restricao.ConstrName}:")
  print(f"  lower bound do recurso (SARHSLow): {restricao.SARHSLow}")
  print(f"  upper bound do recurso (SARHSUp): {restricao.SARHSUp}")


Intervalos de Custos (Vetor de Custos):
Variável McDuplo:
  lower bound do custo (SAObjLow): 2.802995391705069
  upper bound do custo (SAObjUp): 4.239198064293121
Variável Casquinha Baunilha:
  lower bound do custo (SAObjLow): 0.7619047619047619
  upper bound do custo (SAObjUp): 1.5
Variável Maçã:
  lower bound do custo (SAObjLow): 0.5064935064935066
  upper bound do custo (SAObjUp): 2.303457446808511

Intervalos de Recursos (Vetor de Recursos):
Restrição Min_Energia:
  lower bound do recurso (SARHSLow): -inf
  upper bound do recurso (SARHSUp): 136.69724770642202
Restrição Min_Carb.:
  lower bound do recurso (SARHSLow): 82.1705426356589
  upper bound do recurso (SARHSUp): 296.42857142857144
Restrição Min_Prot.:
  lower bound do recurso (SARHSLow): -inf
  upper bound do recurso (SARHSUp): 163.302752293578
Restrição Min_G. Total:
  lower bound do recurso (SARHSLow): -inf
  upper bound do recurso (SARHSUp): 214.67889908256882
Restrição Min_G. Sat.:
  lower bound do recurso (SARHSLow): -i

### Vetor de Custos

Se o custo de alguma variável básica ultrapassar o limite inferior ou superior calculado, então a variável irá sair da base.

### Vetor de Recursos

Em relação ao limitantes inferiores com infinito, não importa o quanto o VDR mínimo do recurso diminua, a solução ótima sempre será a mesma.

Aos demais valores calculados, se algum recurso ultrapassar o limite inferior ou superior do seu VDR mínimo, então alguma variável básica irá se tornar negativa, implicando uma não factibilidade ao problema, logo o PL deve ser resolvido novamente.

## 3) Qual a dieta ótima vegetariana?

Para descobrir a dieta ótima vegetariana há duas maneiras de obter a nova solução:
1. Inserir uma variável binária no modelo para representar se o alimento é vegetariano ou não;
2. Remover todos os alimentos marcados como não vegetarianos nas tabelas em si.

Será executado a maneira (1).

### Modificando o modelo

\begin{equation}
\sum_{j}^n c_{j}x_{j}
\end{equation}

sujeito a:

\begin{alignat}{2}
\sum_{j}^n a_{ji}x_{j} \ge r_{i}, \quad \forall i \in \{1,2,...,m\}. \\
\sum_{j}^n a_{ji}x_{j} \le s_{i}, \quad \forall i \in \{1,2,...,m\}. \\
x_{j} \ge f_{min,j} * y_{j} \quad \forall j \in \{1,2,...,n\} \\
x_{j} \le f_{max,j} * y_{j} \quad \forall j \in \{1,2,...,n\}
\end{alignat}

Se um alimento não for vegetariano, sua variável binária é forçada a 0, impedindo seu consumo.

### Declaração do objeto que representa o modelo matemático

In [ ]:
modelo_veg = Model("MacDonalds_Dieta_Vegetariana")

### Variável de decisão

In [ ]:
x = {j: modelo_veg.addVar(name=f"{alimentos_nomes[j]}", vtype=GRB.CONTINUOUS, lb=lim_inf_compra[j], ub=lim_sup_compra[j]) for j in range(len(alimentos_nomes))}
y = {j: modelo_veg.addVar(name=f"y_{alimentos_nomes[j]}", vtype=GRB.BINARY) for j in range(len(alimentos_nomes))}

### Função objetivo

In [ ]:
modelo_veg.setObjective(quicksum(custos[j] * x[j] for j in range(len(alimentos_nomes))), GRB.MINIMIZE)

### Restrições

In [ ]:
# Restrições nutricionais
for i, nutriente in enumerate(nutrientes_nomes):
  modelo_veg.addConstr(quicksum(coeficientes[j][i] * x[j] for j in range(len(alimentos_nomes))) >= vdr_min[i], f"Min_{nutriente}")
  if vdr_max[i] < float('inf'): # Apenas adiciona a restrição de máximo se existir (diferente de infinito)
    modelo_veg.addConstr(quicksum(coeficientes[j][i] * x[j] for j in range(len(alimentos_nomes))) <= vdr_max[i], f"Max_{nutriente}")

# Restrições considerando as variáveis binárias
for j in range(len(alimentos_nomes)):
  # Se o limitante superior for infinito, define-se uma constante "grande" (ninguém gasta mais que 300 reais no MacDonalds, eu acho...)
  max_bound = lim_sup_compra[j] if lim_sup_compra[j] < float('inf') else 300
  modelo_veg.addConstr(x[j] <= max_bound * y[j], f"MaxUse_{alimentos_nomes[j]}")
  modelo_veg.addConstr(x[j] >= lim_inf_compra[j] * y[j], f"MinUse_{alimentos_nomes[j]}")

for j in range(len(alimentos_nomes)):
  if eh_vegano[j] == False:
    modelo_veg.addConstr(y[j] == 0, f"NonVeg_{alimentos_nomes[j]}")

### Resolvendo o problema

In [ ]:
modelo_veg.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 195 rows, 168 columns and 903 nonzeros
Model fingerprint: 0x34e423f0
Variable types: 84 continuous, 84 integer (84 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective 146.9122995
Presolve removed 185 rows and 131 columns
Presolve time: 0.00s
Presolved: 10 rows, 37 columns, 317 nonzeros
Found heuristic solution: objective 124.0971734
Variable types: 37 continuous, 0 integer (0 binary)

Root relaxation: objective 3.963415e+01, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbe

### Imprimindo as soluções do problema

In [ ]:
if modelo_veg.status == GRB.OPTIMAL:
  print("\nSolução ótima encontrada!")
  for j in range(len(alimentos_nomes)):
    if x[j].x > 1e-6:  # Apenas exibe valores significativos
      print(f"{x[j].varName}: {x[j].x:.2f} unidades")
  print(f"Custo total: R$ {modelo_veg.objVal:.2f}")
else:
    print("Nenhuma solução ótima encontrada.")


Solução ótima encontrada!
Molho Salada Ranch: 7.32 unidades
Casquinha Baunilha: 13.66 unidades
Casquinha Chocolate: 2.20 unidades
Maçã: 4.88 unidades
Custo total: R$ 39.63


## 4) Qual a dieta ótima em que cada alimento é consumido uma única vez?

Para realizar essa nova solução, pode-se modificar f_max do arquivo McDonalds-food.wsv para 1.

Como o f_min (default) sempre é 0, então cada alimento será consumido uma única vez, por partes ou não será consumido. Por precaução, define-se f_min como 0, caso ele não seja default.

### Modelo

\begin{equation}
\sum_{j}^n c_{j}x_{j}
\end{equation}

sujeito a:

\begin{alignat}{2}
\sum_{j}^n a_{ji}x_{j} \ge r_{i}, \quad \forall i \in \{1,2,...,m\}. \\
\sum_{j}^n a_{ji}x_{j} \le s_{i}, \quad \forall i \in \{1,2,...,m\}. \\
x_{j} \ge f_{min,j} \quad \forall j \in \{1,2,...,n\} \\
x_{j} \le f_{max,j} \quad \forall j \in \{1,2,...,n\}
\end{alignat}

$f_{min,j}$ sempre é 0. \\
$f_{max,j}$ sempre é 1.

### Declaração do objeto que representa o modelo matemático

In [ ]:
modelo_unico = Model("MacDonalds_Dieta_Unica")

#### Substituindo valores

In [ ]:
alimentos['Compra_min'] = 0
alimentos['Compra_max'] = 1

lim_inf_compra = alimentos['Compra_min'].values
lim_sup_compra = alimentos['Compra_max'].values

### Variável de decisão

In [ ]:
x = {j: modelo_unico.addVar(name=f"{alimentos_nomes[j]}", vtype=GRB.CONTINUOUS, lb=lim_inf_compra[j], ub=lim_sup_compra[j]) for j in range(len(alimentos_nomes))}

### Função objetivo

In [ ]:
modelo_unico.setObjective(quicksum(custos[j] * x[j] for j in range(len(alimentos_nomes))), GRB.MINIMIZE)

### Restrições

In [ ]:
for i, nutriente in enumerate(nutrientes_nomes):
  modelo_unico.addConstr(quicksum(coeficientes[j][i] * x[j] for j in range(len(alimentos_nomes))) >= vdr_min[i], f"Min_{nutriente}")
  if vdr_max[i] < float('inf'): # Apenas adiciona a restrição de máximo se existir (diferente de infinito)
    modelo_unico.addConstr(quicksum(coeficientes[j][i] * x[j] for j in range(len(alimentos_nomes))) <= vdr_max[i], f"Max_{nutriente}")

### Resolvendo o problema

In [ ]:
modelo_unico.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 84 columns and 634 nonzeros
Model fingerprint: 0xf5f3b527
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 0 rows and 2 columns
Presolve time: 0.02s
Presolved: 10 rows, 82 columns, 634 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.500000e+01   0.000000e+00      0s
       6    3.1562785e+01   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.03 seconds (0.00 work units)
Optimal objective  3.156278455e+01


### Imprimindo as soluções do problema

In [ ]:
if modelo_unico.status == GRB.OPTIMAL:
  print("\nSolução ótima encontrada!")
  for j in range(len(alimentos_nomes)):
    if x[j].x > 1e-6:  # Apenas exibe valores significativos
      print(f"{x[j].varName}: {x[j].x:.2f} unidades")
  print(f"Custo total: R$ {modelo_unico.objVal:.2f}")
else:
    print("Nenhuma solução ótima encontrada.")


Solução ótima encontrada!
Quarterão com Queijo: 1.00 unidades
McDuplo: 1.00 unidades
Cheeseburger: 1.00 unidades
Hamburger: 1.00 unidades
McBacon Junior: 1.00 unidades
McFritas grande: 1.00 unidades
McFritas média: 0.43 unidades
Chicken McNuggets (12 unid.): 0.42 unidades
Casquinha Baunilha: 0.24 unidades
Casquinha Chocolate: 1.00 unidades
Maçã: 1.00 unidades
Custo total: R$ 31.56
